In [ ]:
!pip install opendatasets
!pip install pandas
import opendatasets as od
  
od.download(
    "https://www.kaggle.com/datasets/wcukierski/enron-email-dataset")

In [ ]:
# Load the CSV
import pandas

df = pandas.read_csv('/notebooks/enron-email-dataset/emails.csv')


In [ ]:
import numpy
import email
!pip install quotequail
import quotequail as qq
import random


def formatmsg(msg):
    msg = email.message_from_string(msg).get_payload()
    msg = msg.split('--')[0].strip().split('\n\n\n')[0]
    if len(msg) < 1: return numpy.NAN
    if msg.startswith('-'):
        return numpy.NAN
    elif msg.startswith('http'):
        return numpy.NAN
    return msg
# print(formatmsg(df.loc[random.randint(0,100000)]['message']))



In [ ]:
df['file'] = df['file'].map(lambda x: x.split('/',1)[0])


In [ ]:
#Order by number of contributions by author
df = (df.assign(new =df['file'].map(df['file'].value_counts()))
         .sort_values(['new','file'], ascending=[False, True])
         .drop('new', axis=1))


In [ ]:
#Remove bots
df = df[df["file"].str.contains("auto")==False]

In [ ]:
#Limit the data
num_authors = 70
num_commits_per_author = 500 #≤4000
seenauthors = []
i = 0

# df = df.reset_index()

# print(df.drop_duplicates(subset = "file"))
topauths = df.reset_index().drop_duplicates(subset = "file").iloc[[num_authors]].index[0]

print(topauths)
# print(df.reset_index().drop_duplicates(subset = "file").head())
print(df.reset_index().head(topauths).drop_duplicates(subset='file').drop('index',axis=1).shape[0])

df = df.reset_index().head(topauths)


In [ ]:
df.shape[0]

In [ ]:
# print(df.head(10)['message'].map(lambda x:formatmsg(x)))

df['message'] = df['message'].map(lambda x:formatmsg(x))
df = df.dropna()


In [ ]:
df = df.groupby("file").head(num_commits_per_author).reset_index(drop=True).head(num_authors*num_commits_per_author)
df.shape[0]

In [ ]:
#Export to files
import os

for i, row in df.iterrows():
     testrain = "test" if i%2==0 else "train"
    
     path = f"enron_{num_authors}_{num_commits_per_author}/{testrain}/{row['file']}"

     if not os.path.exists(path):
       os.makedirs(path)

     filename = f"{i}.txt"
     f = open(f"{os.path.join(path, filename)}", "w")
     f.write(str((row['message'])))
     f.close()
    
     if i%100==0:print(i)